In [8]:
!pip install statsmodels

In [9]:
!pip install scikit-learn

In [7]:
conda install xgboost --force conda



         See 'conda update --help' for details about the --force-reinstall
         and --clobber flags.


Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/osx-arm64::conda==22.9.0=py39hca03da5_0
  - defaults/osx-arm64::sip==6.7.12=py39h313beb8_0
  - defaults/osx-arm64::anaconda-navigator==2.5.2=py39hca03da5_0
  - defaults/osx-arm64::clyent==1.2.2=py39hca03da5_1
  - defaults/osx-arm64::conda-repo-cli==1.0.75=py39hca03da5_0
  - defaults/osx-arm64::anaconda-client==1.12.2=py39hca03da5_0
  - defaults/osx-arm64::pyqt==5.15.10=py39h313beb8_0
  - defaults/osx-arm64::navigator-updater==0.4.0=py39hca03da5_1
  - defaults/noarch::conda-token==0.4.0=pyhd3eb1b0_0
done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 24.1.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col
from math import sqrt
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso, LassoCV
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.preprocessing import normalize
import scipy.cluster.hierarchy as shc
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

It's unclear whether or not initializing a random seed will be of use yet, but I'm leaving it here for now.

In [3]:
import random

random.seed(1680)

Use pandas to read in the CSVs to get the relevant dataframes.

In [ ]:
# Read the cab rides and weather data
cab_df = pd.read_csv("cab_rides.csv")
weather_df = pd.read_csv("weather.csv")

The data for Uber and Lyfts has the following variables: distance, cab_type, time_stamp, destination, location, price, surge_multiplier, id, product_id, name, and date_time. 

In [16]:
print(cab_df.head())

   distance cab_type          time_stamp    destination          location  \
0      0.44     Lyft 2018-12-16 04:00:00  North Station  Haymarket Square   
1      0.44     Lyft 2018-11-26 21:00:00  North Station  Haymarket Square   
2      0.44     Lyft 2018-11-27 20:00:00  North Station  Haymarket Square   
3      0.44     Lyft 2018-11-29 23:00:00  North Station  Haymarket Square   
4      0.44     Lyft 2018-11-28 22:00:00  North Station  Haymarket Square   

   price  surge_multiplier                                    id  \
0    5.0               1.0  424553bb-7174-41ea-aeb4-fe06d4f4b9d7   
1   11.0               1.0  4bd23055-6827-41c6-b23b-3c491f24e74d   
2    7.0               1.0  981a3613-77af-4620-a42a-0c0866077d1e   
3   26.0               1.0  c2d88af2-d278-4bfd-a8d0-29ca77cc5512   
4    9.0               1.0  e0126e1f-8ca9-4f2e-82b3-50505a09db9a   

     product_id          name                     date_time  
0     lyft_line        Shared 2018-12-16 09:30:07.890000128  
1  l

The data for weather has the following variables: temp, location, clouds, pressure, rain, time_stamp. Since data was only collected during one week in November, I am inclined not to use the temp variable. In addition, it may overlap with a 'late_night' variable that I am planning to use because low temperatures are related to the time of day, causing multicollinearity. For that reason, I will only be using the 

In [17]:
print(weather_df.head())

    temp            location  clouds  pressure    rain          time_stamp  \
0  42.42            Back Bay     1.0   1012.14  0.1228 2018-12-16 18:00:00   
1  42.43         Beacon Hill     1.0   1012.15  0.1846 2018-12-16 18:00:00   
2  42.50   Boston University     1.0   1012.15  0.1089 2018-12-16 18:00:00   
3  42.11              Fenway     1.0   1012.13  0.0969 2018-12-16 18:00:00   
4  43.13  Financial District     1.0   1012.14  0.1786 2018-12-16 18:00:00   

   humidity   wind           date_time  
0      0.77  11.25 2018-12-16 23:45:01  
1      0.76  11.32 2018-12-16 23:45:01  
2      0.76  11.07 2018-12-16 23:45:01  
3      0.77  11.09 2018-12-16 23:45:01  
4      0.75  11.49 2018-12-16 23:45:01  


pd.to_datetime can help to handle the current date/time variables which are stored in epochs. The cab_df dataframe has the epochs multiplied by 1000.

In [9]:
weather_df['time_stamp']=[datetime.fromtimestamp(i) for i in weather_df['time_stamp']]
weather_df['time_stamp']=weather_df['time_stamp'].values.astype('datetime64[h]')

cab_df['time_stamp']=[datetime.fromtimestamp(i/1000.0) for i in cab_df['time_stamp']]
cab_df['time_stamp']=cab_df['time_stamp'].values.astype('datetime64[h]')
cab_df.rename(columns={'source':'location'},inplace=True)

weather_df.drop_duplicates(['time_stamp','location'],inplace=True)

Merge dataframes on location and time using pd.merge.

In [ ]:
merged_df=pd.merge(cab_df,weather_df,on=['location','time_stamp'])

There are missing values in the price column so we fill them with the mean price for all of the prices.

In [ ]:
merged_df['price'].fillna(merged_df['price'].mean(), inplace=True)

We also fill missing rain values with 0. 

In [ ]:
merged_df['rain'].fillna(0, inplace=True)

Here, we need to normalize the price. Uber and Lyft rides have different distances and durations which factors into the cost. Sadly, ride duration is not an available variable, but we can divide the distance by the price to get an approximately normalized price.  

In [ ]:
# Normalize price data by dividing price by distance
merged_df['normalized_price'] = merged_df['price'] / merged_df['distance']

To determine when prices are high, a lot of time-based variables are necessary. It is not useful to include every day of the week as a variable because of multicollinearity issues, so instead, dummy variables will be used to single out specific windows of time that do not overlap. However, I will add a graph here to show the relationship between time and cab prices.

In [ ]:
merged_df['day_of_week'] = merged_df['time_stamp'].dt.dayofweek

Here is the code for rush hour. Rush hour is the period from 8 AM to noon and 4 PM to 6 PM. I am only counting rush hour on weekdays to avoid overlap with the weekend variable.

In [ ]:
merged_df['rush_hour'] = ((merged_df['time_stamp'].dt.hour >= 8) & (merged_df['time_stamp'].dt.hour < 12) |
                          ((merged_df['time_stamp'].dt.hour >= 16) & (merged_df['time_stamp'].dt.hour < 18))).astype(int)

Late night is every single day from 10 PM to 6 AM.

In [ ]:
merged_df['late_night'] = ((merged_df['time_stamp'].dt.hour >= 22) | (merged_df['time_stamp'].dt.hour < 6)).astype(int)

Weekend is Fridays after 6 PM and until 10 PM, Saturdays from 6 AM until 10PM, and Sundays from 6 AM until 10PM.

In [ ]:
merged_df['weekend'] = ((merged_df['time_stamp'].dt.dayofweek == 4) & (merged_df['time_stamp'].dt.hour >= 18)) | \
                       (merged_df['time_stamp'].dt.dayofweek == 5) | \
                       ((merged_df['time_stamp'].dt.dayofweek == 6) & (merged_df['time_stamp'].dt.hour < 22)).astype(int)

merged_df['weekend'] = merged_df['weekend'].astype(float)

This is the dummy variable for heavy rain. Heavy rain is defined for now as any period where there is more than 0.1 inch in an hour. 

In [ ]:
merged_df['heavy_rain'] = (merged_df['rain'] > 0.1).astype(int)

This is the code for an OLS regression. I have coded up multiple values of X and Y to show the differences between the effects of different variables on X and Y.

In [19]:
X1 = merged_df[['rush_hour', 'late_night', 'weekend', 'heavy_rain']]
X2 = merged_df[['rush_hour']]
y1 = merged_df['normalized_price']
y2 = merged_df['surge_multiplier']

X = sm.add_constant(X)

model1 = sm.OLS(y1, X1).fit()

print(model.summary())
#print(weather_df.shape[0])
#print(merged_df.head(100))

                            OLS Regression Results                            
Dep. Variable:       normalized_price   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.2645
Date:                Fri, 08 Mar 2024   Prob (F-statistic):              0.901
Time:                        20:10:11   Log-Likelihood:            -2.8173e+06
No. Observations:              690107   AIC:                         5.635e+06
Df Residuals:                  690102   BIC:                         5.635e+06
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          9.8260      0.027    362.962      0.0

In [45]:
import xgboost as xg

X = merged_df[['rush_hour', 'late_night', 'day_of_week']]
y = merged_df['surge_multiplier']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xgb_model = xgb.XGBRegressor()

xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

ModuleNotFoundError: No module named 'xgboost'